# Numpy

¿Que vamos a ver hoy?

- ND-Array
- dtype
- index/slicing (repaso)
- mascaras e indices booleanos
- operaciones vectoriales (no vamos a ver matematica hoy)
- Reglas para extender arrays en operaciones
- Que es un seed
- idea de ufunc
  - sobre un vector
  - sobre dos vectores
  - agregacion
- reduccion con axis
- leer archivos genfromtxt
- cambiar tipos astype

## Funciones especificas

- array
- shape
- size
- ndim
- arange
- ones
- zeros
- reshape
- random
- random.uniform
- empty


## otras cosas

- operador %

# Problema a solucionar

Tenemos un archivo excel con información sobre inversiones e inflación. Queremos saber que inversiones le ganaron a la inflación y cuales no. 

Vamos a suponer las siguientes cuestiones:

- Lo que se cobra como cupon se descuenta del valor del bono
- La plata que se cobra en cupon o interes se guarda en algun activo que ajusta por inflacion. 

![Inversion](./Inversion.png)

En este problema, tenemos que solucionar los siguientes problemas:

- Leer la información.
- Poder hacer cuentas mas o menos complejas (vamos a ver cuales, no hace falta que las inventen ustedes) financieras repitiendo operaciones para los datos.

## Pasos a realizar

- Leer la informacion correctamente y separar los encabezados del contenido.
- Separar la informacion segun la columna 
- Construir un índice de inflacion a partir de los porcentajes de inflación mensual. 
- Calcular cuanto valdrian las inversiones si se colocara en algun activo que se ajuste por inflación.
- Incluir el ajuste de valor de los cupones e interes. 
- Calcular cuanto valen las inversiones al finalizar el periodo para poder comparar con la inflación, incluyendo los cupones e interes.

In [9]:
# importamos numpy

import numpy as np

In [14]:
filename = "./Datos.csv"

data_cruda = np.genfromtxt(filename, delimiter="\t",dtype = str) # Notar que si no ponemos el separador correcto tira error. Y si no ponemos dtype lee todo como vacio.

data_cruda

array([['', 'Enero', '', '', 'Febrero', '', '', 'Marzo', '', '', 'Abril',
        '', '', 'Mayo', '', '', 'Junio', '', '', 'Julio', '', '',
        'Agosto', '', '', 'Septiembre', '', '', 'Octubre', '', '',
        'Noviembre', '', '', 'Diciembre', '', ''],
       ['', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)'],
       ['Inflacion (%)', '', '2', '', '', '3.1', '', '', '1.9', '', '',
        '1.4', '', '', '1.5', '', '', '1.8', '', '', '2.2', '', '',
        '1.6', '', '', '1.2', '', '', '1', '', '', '0.9', '', '', '1.2',
        ''],
       ['ETH', '1000', '123', '', '', '130', '', '', '145',

In [29]:
data_cruda[0]
data_cruda[1]


lineas_de_encabezado = 2

encabezados = data_cruda[:lineas_de_encabezado]
inflacion = data_cruda[lineas_de_encabezado]
contenido = data_cruda[lineas_de_encabezado + 1:]

In [48]:
# Queremos armar una lista de los meses

print (encabezados)

meses = encabezados[0,1:encabezados[0].size:3]

[['' 'Enero' '' '' 'Febrero' '' '' 'Marzo' '' '' 'Abril' '' '' 'Mayo' ''
  '' 'Junio' '' '' 'Julio' '' '' 'Agosto' '' '' 'Septiembre' '' ''
  'Octubre' '' '' 'Noviembre' '' '' 'Diciembre' '' '']
 ['' 'Nominales' 'Valor' 'C&I (U)' 'Nominales' 'Valor' 'C&I (U)'
  'Nominales' 'Valor' 'C&I (U)' 'Nominales' 'Valor' 'C&I (U)' 'Nominales'
  'Valor' 'C&I (U)' 'Nominales' 'Valor' 'C&I (U)' 'Nominales' 'Valor'
  'C&I (U)' 'Nominales' 'Valor' 'C&I (U)' 'Nominales' 'Valor' 'C&I (U)'
  'Nominales' 'Valor' 'C&I (U)' 'Nominales' 'Valor' 'C&I (U)' 'Nominales'
  'Valor' 'C&I (U)']]


In [66]:
# Queremos de contenido extraer los valores de "Nominal"
contenido
encabezados[1]

texto = "Nominales"
mascara_indices_nominal = encabezados[1] == texto

texto = "Valor"
mascara_indices_valor = encabezados[1] == texto

texto = "C&I (U)"
mascara_indices_cupon = encabezados[1] == texto

contenido[0][mascara_indices_valores]

# Que tipo de abjeto es mascara...

type(mascara_indices_valores)


numpy.ndarray

In [71]:
# Ahora vamos a extraer la informacion en forma de diccionario para cada activo

activos = []

# contenido[contenido == ""] = np.nan
contenido[contenido == ""] = "0"

for elemento in contenido:
    #print (elemento)
    activo = {}
    activo["Nombre"] = elemento[0]
    activo["Valores"] = elemento[mascara_indices_valor].astype(np.float)
    activo["Cupones"] = elemento[mascara_indices_cupon].astype(np.float)
    activo["Nominales"] = elemento[mascara_indices_nominal].astype(np.float)
    activos.append(activo)

print (activos)

[{'Nombre': 'ETH', 'Valores': array([123., 130., 145., 150., 110., 105., 132., 148., 170., 185., 156.,
       140.]), 'Cupones': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Nominales': array([1000.,    0.,    0.,    0.,    0.,    0.,    0., -500.,    0.,
          0.,    0., -500.])}, {'Nombre': 'GOLD', 'Valores': array([  0., 110., 111., 132., 112., 115., 109.,  85.,  68.,  45.,  22.,
         0.]), 'Cupones': array([ 0.,  5.,  5.,  5.,  5.,  5.,  5., 20., 20., 20., 20., 20.]), 'Nominales': array([  0.,  73.,  10.,  50., -80.,   0., -53.,   0.,   0.,   0.,   0.,
         0.])}, {'Nombre': 'Al29', 'Valores': array([54., 57., 60., 62., 65., 69., 75., 72., 70., 73., 74., 77.]), 'Cupones': array([0., 0., 0., 0., 0., 0., 0., 7., 3., 7., 3., 7.]), 'Nominales': array([ 300.,    0., -150.,    0.,  200.,    0., -150.,    0., -100.,
          0., -100.,    0.])}]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  del sys.path[0]
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: `np.float` is a dep

In [84]:
# Hasta aca con listas se podria haber hecho casi que lo mismo.

# Vamos a calcular la inflacion acumulada desde cada mes hasta el final. 

# Veamos primero como es la info que tenemos de inflacion.

inflacion

inflacion_numerica = inflacion[mascara_indices_valor].astype(np.float)

print (inflacion_numerica)

inflacion_numerica = inflacion_numerica/100 + 1

print (inflacion_numerica)


inflacion_acumulada = inflacion_numerica.cumprod()

inflacion_acumulada

# Esto sirve si tenemos un valor en Enero y queremos saber cuando va a valer por ejemplo hasta Julio, hacemos el valor en enero por el valor del casillero Julio, pero no sirve si queremos hacer el valor en diciembre de algo que tenemos en Julio
# Para esta segunda cuenta necesitamos multiplicar por la inflacion mensual desde atras. 

inflacion_acumulada_reversa = np.flip(np.flip(inflacion_numerica).cumprod())

inflacion_acumulada_reversa


[2.  3.1 1.9 1.4 1.5 1.8 2.2 1.6 1.2 1.  0.9 1.2]
[1.02  1.031 1.019 1.014 1.015 1.018 1.022 1.016 1.012 1.01  1.009 1.012]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


array([1.21675451, 1.19289658, 1.15702869, 1.13545504, 1.11977815,
       1.1032297 , 1.0837227 , 1.06039403, 1.04369491, 1.03131908,
       1.021108  , 1.012     ])

In [97]:
activo = activos[2]
activo

print (activo["Nominales"])

activo["Nominales_totales"] = activo["Nominales"].cumsum()
print(activo["Nominales_totales"])

activo["Beneficio_mes_a_mes"] = activo["Nominales_totales"] * activo["Cupones"]

print (activo["Beneficio_mes_a_mes"])

activo["Beneficio_mes_a_mes_valuado_hoy"] = activo["Beneficio_mes_a_mes"] * inflacion_acumulada_reversa
print (activo["Beneficio_mes_a_mes_valuado_hoy"])

activo["Beneficio_valuado_hoy_total"] = activo["Beneficio_mes_a_mes_valuado_hoy"].sum()
print (activo["Beneficio_valuado_hoy_total"])


[ 300.    0. -150.    0.  200.    0. -150.    0. -100.    0. -100.    0.]
[300. 300. 150. 150. 350. 350. 200. 200. 100. 100.   0.   0.]
[   0.    0.    0.    0.    0.    0.    0. 1400.  300.  700.    0.    0.]
[   0.            0.            0.            0.            0.
    0.            0.         1484.5516385   313.10847269  721.923356
    0.            0.        ]
2519.5834671927037


In [99]:
print (activo["Valores"])
print (activo["Nominales"])

# Vamos a modificar la lista de nominales para que "se venda" todo en el ultimo mes.

activo["Nominales_liquidado"] = activo["Nominales"]
activo["Nominales_liquidado"][-1] = activo["Nominales_liquidado"][-1] + - activo["Nominales"].sum()
print (activo["Nominales_liquidado"])


activo["CambioValor_mes_a_mes_valudado_hoy"] = - activo["Nominales_liquidado"]*activo["Valores"]*inflacion_acumulada_reversa # El menos es porque un nominal positivo es comprar pero es un gasto.

print (activo["CambioValor_mes_a_mes_valudado_hoy"])

activo["CambioValor_valuado_hoy"] = activo["CambioValor_mes_a_mes_valudado_hoy"].sum()

print (activo["CambioValor_valuado_hoy"])

print (activo["Beneficio_valuado_hoy_total"])
print ("Ganancia neta valudad hoy: " + str(activo["CambioValor_valuado_hoy"] + activo["Beneficio_valuado_hoy_total"]))


[54. 57. 60. 62. 65. 69. 75. 72. 70. 73. 74. 77.]
[ 300.    0. -150.    0.  200.    0. -150.    0. -100.    0. -100.    0.]
[ 300.    0. -150.    0.  200.    0. -150.    0. -100.    0. -100.    0.]
[-19711.42308044     -0.          10413.25821138     -0.
 -14557.1159527      -0.          12191.88033122     -0.
   7305.86436272     -0.           7556.1992         -0.        ]
3198.663072168253
2519.5834671927037
Ganancia neta valudad hoy: 5718.246539360956


In [103]:
a = np.array([1,2,3])
b = np.array([1,2,4])

a == b

c = [1,2,3]
d = [1,2,4]

c == d


a == 2

array([False,  True, False])

In [105]:
a = np.array([[1,2],[3,4]])
a

array([[1, 2],
       [3, 4]])

In [108]:
a.sum(axis=1)

array([3, 7])